In [1]:
# Importing the required libraries

import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

In [3]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot.h1')

Function to clean the sentence or user input

In [4]:
def clean_up_sentence(sentence):
    
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

Function to create bow (bag of words) using the clean sentence from the above step

In [15]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details = True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return (np.array(bag))

Function to predict the target class

In [16]:
def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details = False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > error]
    
    # sort by strength of probability
    results.sort(key = lambda x:x[1], reverse = True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

Function to get responses from the model

In [17]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

Function to start the chatbot which will run till the user type 'end'

In [18]:
def start_chat():
    print('Bot: This is Ted! Your Personal Assistant.\n\n')
    while True:
        inp = str(input()).lower()
        if inp.lower() == "end":
            break
        if inp.lower() == '' or inp.lower() == '*':
            print('Please re-phrase your query!')
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [19]:
# Start the chat
start_chat()

Bot: This is Ted! Your Personal Assistant.


hey
1/1 [==============================] - 0s 17ms/step
Bot: Good to see you again

--------------------------------------------------
what you do
1/1 [==============================] - 0s 18ms/step
Bot: I am a general purpose chatbot. My capabilities are : 
 1. I can chat with you. Try asking me for jokes or riddles! 
 2. Ask me the date and time 
 3. I can google search for you. Use format google: your query 
 4. I can get the present weather for any city. Use format weather: city name 
 5. I can get you the top 10 trending news in India. Use keywords 'Latest News' 
 6. I can get you the top 10 trending songs globally. Type 'songs' 
 7. I can set a timer for you. Enter 'set a timer: minutes to timer' 
 8. I can get the present Covid stats for any country. Use 'covid 19: world' or 'covid 19: country name' 
 For suggestions to help me improve, send an email to ted.thedlbot.suggestions@gmail.com . Thank you!! 

-------------------------------

In [20]:
# Turning the Chatbot into a GUI with tkinter

import random
import tkinter as tk
from tkinter import *

root = tk.Tk()
filename = "Chat Bot"
root.title(f"Chat Bot")
root.geometry('500x400')
root.resizable(False, False)
message = tk.StringVar()

chat_win = Frame(root, bd = 1, bg = 'white', width = 50, height = 8)
chat_win.place(x = 6, y = 6, height = 300, width = 488)

textcon = tk.Text(chat_win, bd = 1, bg = 'white', width = 50, height = 8)
textcon.pack(fill = 'both', expand = True)

mes_win = Entry(root, width = 30, xscrollcommand = True, textvariable = message)
mes_win.place(x=6, y=310, height=60, width=380)
mes_win.focus()

textcon.config(fg='black')
textcon.tag_config('usr', foreground = 'black')
textcon.insert(END, "Bot: This is Ted! Your Personal Assistant.\n\n")
mssg = mes_win.get()

exit_list = ['exit', 'break', 'quit', 'see you later', 'chat with you later', 'end the chat', 'bye', 'ok bye']

def greet_res(text):
    text = text.lower()
    bot_greet = ['hi', 'hello', 'hola', 'hey', 'howdy']
    usr_greet = ['hi', 'hey', 'hello', 'hola', 'greetings', 'wassup', 'whatsup']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

def send_msz(event = None):
    usr_input = message.get()
    usr_input = usr_input.lower()
    textcon.insert(END, f'You: {usr_input}'+'\n', 'usr')
    if usr_input in exit_list:
        textcon.config(fg = 'black')
        textcon.insert(END, "Bot: Ok bye! Chat with you later\n")
        return root.destroy()
    else:
        textcon.config(fg = 'black')
        if greet_res(usr_input) != None:
            lab = f"Bot: {greet_res(usr_input)}"+'\n'
            textcon.insert(END, lab)
            mes_win.delete(0, END)
        else:
            lab = f"Bot: {chatbot_response(usr_input)}"+'\n'
            textcon.insert(END, lab)
            mes_win.delete(0, END)

button_send = Button(root, text = 'Send', bg = 'dark green', activebackground = 'grey',
                    command = send_msz, width = 12, height = 5, font = ('Arial'))
button_send.place(x = 376, y = 310, height = 60, width = 110)
root.bind('<Return>', send_msz, button_send)
root.mainloop()

1/1 [==============================] - 0s 15ms/step
